In [1]:
#!/usr/bin/env python
#SBATCH --job-name=PSNN
#SBATCH --error=%x.%j.err
#SBATCH --output=%x.%j.out
#SBATCH --mail-user=hzhao@teco.edu
#SBATCH --export=ALL
#SBATCH --time=48:00:00
#SBATCH --partition=sdil
#SBATCH --gres=gpu:1

import os
import sys
sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), 'utils'))
from configuration import *
import torch
import pprint
from utils import *
import BaselineModels as B

In [2]:
args = parser.parse_args([])
args.DEVICE = 'cpu'
args.task = 'temporized'
args.SEED = 1
args.metric = 'acc_snn'

In [3]:
args = FormulateArgs(args)
print(f'Training network on device: {args.DEVICE}.')
MakeFolder(args)

Training network on device: cpu.


In [4]:
train_loader, datainfo = GetDataLoader(args, 'train')
valid_loader, datainfo = GetDataLoader(args, 'valid')
test_loader, datainfo = GetDataLoader(args, 'test')
pprint.pprint(datainfo)

{'N_class': 2,
 'N_feature': 6,
 'N_test': 25,
 'N_time': 32,
 'N_train': 70,
 'N_valid': 23,
 'dataname': 'acuteinflammation'}


In [5]:
for x, y in train_loader:
    print(x.shape, y.shape)
for x, y in valid_loader:
    print(x.shape, y.shape)
for x, y in test_loader:
    print(x.shape, y.shape)

torch.Size([70, 6, 32]) torch.Size([70])
torch.Size([23, 6, 32]) torch.Size([23])
torch.Size([25, 6, 32]) torch.Size([25])


In [6]:
SetSeed(args.SEED)
setup = f"baseline_model_SNN_data_{datainfo['dataname']}_seed_{args.SEED}.model"
print(f'Training setup: {setup}.')

Training setup: baseline_model_SNN_data_acuteinflammation_seed_1.model.


In [7]:
msglogger = GetMessageLogger(args, setup)
msglogger.info(f'Training network on device: {args.DEVICE}.')
msglogger.info(f'Training setup: {setup}.')
msglogger.info(datainfo)

In [8]:
if os.path.isfile(f'{args.savepath}/{setup}'):
    print(f'{setup} exists, skip this training.')
    msglogger.info('Training was already finished.')
else:
    topology = [datainfo['N_feature']] + args.hidden + [datainfo['N_class']]
    msglogger.info(f'Topology of the network: {topology}.')

    snn = B.SpikingNeuralNetwork(topology).to(args.DEVICE)

    lossfunction = B.SNNLoss().to(args.DEVICE)
    optimizer = torch.optim.Adam(snn.parameters(), lr=args.LR)

    if args.PROGRESSIVE:
        snn, best = train_pnn_progressive(snn, train_loader, valid_loader, lossfunction, optimizer, args, msglogger, UUID=setup)
    else:
        snn, best = train_pnn(snn, train_loader, valid_loader, lossfunction, optimizer, args, msglogger, UUID=setup)

    if best:
        if not os.path.exists(f'{args.savepath}/'):
            os.makedirs(f'{args.savepath}/')
        torch.save(snn, f'{args.savepath}/{setup}')
        msglogger.info('Training if finished.')
    else:
        msglogger.warning('Time out, further training is necessary.')

| Epoch:      0 | Train loss: 0.7251 | Valid loss: 0.6854 | Train acc: 0.5286 | Valid acc: 0.6087 | patience:   0 | lr: 0.1 | Epoch time: 0.2 | Power: 0.00e+00 |
| Epoch:      1 | Train loss: 0.6936 | Valid loss: 0.6793 | Train acc: 0.5286 | Valid acc: 0.6087 | patience:   0 | lr: 0.1 | Epoch time: 0.1 | Power: 0.00e+00 |
| Epoch:      2 | Train loss: 0.7032 | Valid loss: 0.6422 | Train acc: 0.5286 | Valid acc: 0.6087 | patience:   0 | lr: 0.1 | Epoch time: 0.2 | Power: 0.00e+00 |
| Epoch:      3 | Train loss: 0.6783 | Valid loss: 0.5884 | Train acc: 0.5286 | Valid acc: 0.6087 | patience:   0 | lr: 0.1 | Epoch time: 0.2 | Power: 0.00e+00 |
| Epoch:      4 | Train loss: 0.5861 | Valid loss: 0.6495 | Train acc: 0.5286 | Valid acc: 0.6087 | patience:   1 | lr: 0.1 | Epoch time: 0.2 | Power: 0.00e+00 |
| Epoch:      5 | Train loss: 0.6374 | Valid loss: 0.5925 | Train acc: 0.5286 | Valid acc: 0.7826 | patience:   2 | lr: 0.1 | Epoch time: 0.1 | Power: 0.00e+00 |
| Epoch:      6 | Train loss

In [ ]:


for seed in range(10):

    args.SEED = seed
    
    
    

    

    

    

    

    